<a href="https://colab.research.google.com/github/adamd1985/Lectures_On_MLAI/blob/main/4_7_Ensemble_Lecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ensemble Methods

Ensemble methods combine the predictions of several other machine learning models, where by aggregating the results of the other models trained on the same dataset. These methods are classified into four general groups:

- Voting Methods: make predictions based on the majority label predicted.
  - Bagging Method: train individual models on random subsets, before voting.
  - Boosting Methods: train individual models sequentially by learning from the errors before voting.
- Stacking Methods: train individual models, and use a meta-model to aggregate their predictions.


In [16]:
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

We'll use the breast cancer dataset again in this notebook.

In [17]:
from sklearn.datasets import load_breast_cancer

breast_cancer = load_breast_cancer(as_frame=True)
X, y = breast_cancer.data, breast_cancer.target
breast_cancer.frame.sample(3)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
379,11.08,18.83,73.3,361.6,0.12160,0.21540,0.16890,0.06367,0.2196,0.07950,...,32.82,91.76,508.1,0.2184,0.9379,0.8402,0.2524,0.4154,0.14030,0
182,15.70,20.31,101.2,766.6,0.09597,0.08799,0.06593,0.05189,0.1618,0.05549,...,32.82,129.30,1269.0,0.1414,0.3547,0.2902,0.1541,0.3437,0.08631,0
417,15.50,21.08,102.9,803.1,0.11200,0.15710,0.15220,0.08481,0.2085,0.06864,...,27.65,157.10,1748.0,0.1517,0.4002,0.4211,0.2134,0.3003,0.10480,0


In [18]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

X, y = breast_cancer.data.values, breast_cancer.target.values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Voting Methods in Ensemble Learning

Ensemble learning leverages multiple models to improve predictive performance. A fundamental technique in ensemble learning is **voting**, where multiple classifiers are trained independently, and their predictions are aggregated to make a final decision.

Hard voting, also known as majority voting, involves taking the most frequently predicted class label among all base models. Given $m$ classifiers, each producing a prediction $y_i^j$ for sample $x_i$, the ensemble prediction is determined as:

$$
\hat{y}_i = \text{mode}(y_i^1, y_i^2, ..., y_i^m)
$$

where $\text{mode}(\cdot)$ returns the most common class among all base classifiers.

For binary classification, this can be formally expressed using the indicator function:

$$
\hat{y}_i = \mathbb{I} \left( \frac{1}{m} \sum_{j=1}^{m} y_i^j > 0.5 \right)
$$

where $\mathbb{I}$ is the indicator function that assigns class 1 if the majority votes for it; otherwise, class 0.


Soft voting takes into account the predicted probabilities from each classifier rather than just the final class label. Each classifier outputs a probability distribution over the possible classes:

$$
P(y_i = c | x_i) = \frac{1}{m} \sum_{j=1}^{m} P_j(y_i = c | x_i)
$$

where $P_j(y_i = c | x_i)$ is the probability assigned to class $c$ by the $j$-th model.

The final prediction is then assigned to the class with the highest averaged probability:

$$
\hat{y}_i = \arg\max_{c} \left( \frac{1}{m} \sum_{j=1}^{m} P_j(y_i = c | x_i) \right)
$$

Soft voting generally works better when classifiers output well-calibrated probabilities.

For regression tasks, the final prediction is computed as the average of all base model predictions:

$$
\hat{y}_i = \frac{1}{m} \sum_{j=1}^{m} \hat{y}_i^j
$$

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)

lr_preds = lr_model.predict(X_test)
lr_acc = accuracy_score(y_test, lr_preds)
print('Logistic Regression accuracy is {0:7.4f} %'.format(lr_acc*100))

Logistic Regression accuracy is 95.6140 %


In [20]:
from sklearn.svm import SVC

# Set probability=True to access probas.
svm_model = SVC(probability=True, random_state=1)
svm_model.fit(X_train, y_train)

svm_preds = svm_model.predict(X_test)
svm_acc = accuracy_score(y_test, svm_preds)
print('SVM accuracy is {0:7.4f} %'.format(svm_acc*100))

SVM accuracy is 94.7368 %


In [21]:
from sklearn.tree import DecisionTreeClassifier

tree_model = DecisionTreeClassifier(random_state=1)
tree_model.fit(X_train, y_train)
tree_preds = tree_model.predict(X_test)

tree_acc = accuracy_score(y_test, tree_preds)
print('Decision Tree accuracy is {0:7.4f} %'.format(tree_acc * 100))

Decision Tree accuracy is 93.8596 %


Now let's build a hard and soft voting mechanism.

In [22]:
# hard voting (majority vote)
predictions = np.array([lr_preds, svm_preds, tree_preds])
ensemble_hard_preds = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=predictions)

# soft voting (average probability)
log_reg_probs = lr_model.predict_proba(X_test)
svc_probs = svm_model.predict_proba(X_test)
tree_probs = tree_model.predict_proba(X_test)
avg_probs = (log_reg_probs + svc_probs + tree_probs) / 3
ensemble_soft_preds = np.argmax(avg_probs, axis=1)

hard_acc = accuracy_score(y_test, ensemble_hard_preds)
soft_acc = accuracy_score(y_test, ensemble_soft_preds)
print(f'Hard Voting Classifier accuracy: {hard_acc * 100:.4f}%')
print(f'Soft Voting Classifier accuracy: {soft_acc * 100:.4f}%')

Hard Voting Classifier accuracy: 96.4912%
Soft Voting Classifier accuracy: 95.6140%


We can validate using Scikit learn [VotingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html).

In [23]:
from sklearn.ensemble import VotingClassifier

hard_voting_clf = VotingClassifier(estimators=[('lr', lr_model), ('svc', svm_model), ('tree', tree_model)], voting='hard')
hard_voting_clf.fit(X_train, y_train)
ensemble_hard_preds = hard_voting_clf.predict(X_test)

soft_voting_clf = VotingClassifier(estimators=[('lr', lr_model), ('svc', svm_model), ('tree', tree_model)], voting='soft')
soft_voting_clf.fit(X_train, y_train)
ensemble_soft_preds = soft_voting_clf.predict(X_test)

hard_acc = accuracy_score(y_test, ensemble_hard_preds)
soft_acc = accuracy_score(y_test, ensemble_soft_preds)
print(f'Hard Voting Classifier accuracy: {hard_acc * 100:.4f}%')
print(f'Soft Voting Classifier accuracy: {soft_acc * 100:.4f}%')

Hard Voting Classifier accuracy: 96.4912%
Soft Voting Classifier accuracy: 95.6140%


## Bagging Ensemble

The term **bagging** is short for *bootstrap aggregating*, where bootstrapping refers to sampling subsets from the training data with replacement. Bagging is an ensemble learning technique that aims to reduce variance and improve model stability by training multiple base learners on different random subsets of the dataset and aggregating their predictions.

Given a training dataset of size $N$, bagging creates $m$ bootstrap samples $D_1, D_2, ..., D_m$ by randomly sampling with replacement from the original dataset.

Each base model $f_j(x)$ is trained independently on a bootstrap sample $D_j$ and produces a prediction $\hat{y}_i^j$ for an input sample $x_i$.



In [24]:
from sklearn.utils import resample

n_estimators = 10
tree_models = []
for _ in range(n_estimators):
    X_resampled, y_resampled = resample(X_train, y_train, random_state=1)
    tree = DecisionTreeClassifier(random_state=1)
    tree.fit(X_resampled, y_resampled)
    tree_models.append(tree)

In [25]:
tree_preds = np.array([tree.predict(X_test) for tree in tree_models])
ensemble_hard_preds = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=tree_preds)

tree_probs = np.mean([tree.predict_proba(X_test) for tree in tree_models], axis=0)
ensemble_soft_preds = np.argmax(tree_probs, axis=1)

hard_acc = accuracy_score(y_test, ensemble_hard_preds)
soft_acc = accuracy_score(y_test, ensemble_soft_preds)
print(f'Bagging Hard Voting Classifier accuracy: {hard_acc * 100:.4f}%')
print(f'Bagging Soft Voting Classifier accuracy: {soft_acc * 100:.4f}%')

Bagging Hard Voting Classifier accuracy: 92.9825%
Bagging Soft Voting Classifier accuracy: 92.9825%


Bagging Ensembles often outperform Voting Ensembles, and they can decrease the overfitting of Decision Trees (which tend to overfit easily). As well as, Bagging Ensembles can be trained in parallel using different CPU cores, which can reduce the processing time.  

Random Forest can be considered a Bagging Ensemble trained on the whole dataset if you set max samples to 1, in fact let's check using scikit [RandomForest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html).

In [26]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=10, max_samples=1.0, bootstrap=True, random_state=1)
rf_model.fit(X_train, y_train)

ensemble_hard_preds = rf_model.predict(X_test)
ensemble_soft_preds = np.argmax(rf_model.predict_proba(X_test), axis=1)
hard_acc = accuracy_score(y_test, ensemble_hard_preds)
soft_acc = accuracy_score(y_test, ensemble_soft_preds)
print(f'Random Forest (Bagging) Hard Voting accuracy: {hard_acc * 100:.4f}%')
print(f'Random Forest (Bagging) Soft Voting accuracy: {soft_acc * 100:.4f}%')

Random Forest (Bagging) Hard Voting accuracy: 95.6140%
Random Forest (Bagging) Soft Voting accuracy: 95.6140%


## Boosting Ensemble

Boosting is an ensemble learning technique that trains individual models **sequentially**, where each subsequent model learns from the errors of the previous models and attempts to improve overall performance. Unlike bagging, which trains models independently in parallel, boosting adjusts model weights dynamically to focus on **misclassified samples**.

Given a dataset with $N$ training samples, boosting maintains a weight distribution $w_i$ for each training sample, adjusting it iteratively to give more importance to **misclassified samples**.

Each model $f_j(x)$ is trained sequentially on a weighted version of the dataset $D$, where misclassified instances receive higher weights. The weight update is defined as:

$$
w_i^{(t+1)} = w_i^{(t)} e^{-\alpha_t y_i f_t(x_i)}
$$

where:
- $w_i^{(t)}$ is the weight of sample $i$ at iteration $t$.
- $\alpha_t$ is the model weight, computed as:

  $$
  \alpha_t = \frac{1}{2} \ln \left( \frac{1 - e_t}{e_t} \right)
  $$

  where $e_t$ is the weighted error rate of the weak learner.
- $y_i$ is the true class label.
- $f_t(x_i)$ is the prediction of the weak learner at step $t$.

After training multiple weak models, the final ensemble prediction is obtained by weighted voting:

$$
F(x) = \sum_{t=1}^{m} \alpha_t f_t(x)
$$

For classification, the predicted label is:

$$
\hat{y} = \text{sign}(F(x))
$$

For regression, the final prediction is a weighted sum of weak learner outputs:

$$
\hat{y} = \sum_{t=1}^{m} \alpha_t f_t(x)
$$

## Gradient Boosting

Gradient Boosting is an ensemble learning technique that optimizes predictions by training models sequentially, where each model corrects the errors (residuals) of the previous model. Unlike other boosting methods, Gradient Boosting uses **gradient descent** to minimize a specific loss function, making it particularly effective for both regression and classification tasks.

  The first model $F_0(x)$ is trained to predict the initial estimate, often using the mean value (for regression) or log-odds (for classification). For example, for regression:

  $$
  F_0(x) = \arg\min_c \sum_{i=1}^N L(y_i, c)
  $$

  where $L(y_i, c)$ is the loss function, such as squared error.

  For each subsequent step $m$, the model computes the **pseudo-residuals** (negative gradient of the loss function):

  $$
  r_i^{(m)} = - \frac{\partial L(y_i, F_{m-1}(x_i))}{\partial F_{m-1}(x_i)}
  $$

  These residuals represent how much the current model’s predictions differ from the actual targets.

  A weak learner $h_m(x)$ (usually a decision tree) is then trained to predict these residuals:

  $$
  h_m(x) = \arg\min_h \sum_{i=1}^N \left( r_i^{(m)} - h(x_i) \right)^2
  $$


  The ensemble prediction is updated by adding the scaled weak learner’s predictions:

  $$
  F_m(x) = F_{m-1}(x) + \eta h_m(x)
  $$

  where $\eta$ is the learning rate that controls how much the weak learner contributes to the final model.


  After $M$ steps, the final model is:

  $$
  F(x) = F_0(x) + \sum_{m=1}^M \eta h_m(x)
  $$

  For regression, this is the final predicted value. For classification, the final predictions are converted to probabilities, and the class with the highest probability is chosen.


In [27]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import log_loss, accuracy_score

n_estimators = 50
learning_rate = 0.1
max_depth = 3

# Initialize predictions with log odds
p = np.mean(y_train)
F_train = np.full_like(y_train, np.log(p / (1 - p)), dtype=np.float64)
F_test = np.full_like(y_test, np.log(p / (1 - p)), dtype=np.float64)

for _ in range(n_estimators):
    # Compute pseudo residuals
    p_train = 1 / (1 + np.exp(-F_train))
    residuals = y_train - p_train

    tree = DecisionTreeRegressor(max_depth=max_depth)
    tree.fit(X_train, residuals)

    tree_preds = tree.predict(X_train)
    numerator = np.sum(residuals * tree_preds)
    denominator = np.sum(tree_preds * tree_preds)
    gamma = numerator / denominator if denominator != 0 else 0

    F_train += learning_rate * gamma * tree.predict(X_train)
    F_test += learning_rate * gamma * tree.predict(X_test)
ensemble_probs = 1 / (1 + np.exp(-F_test))  # Convert log-odds to probability
ensemble_preds = (ensemble_probs >= 0.5).astype(int)

logloss = log_loss(y_test, ensemble_probs)
accuracy = accuracy_score(y_test, ensemble_preds)
print(f'Gradient Boosting Log Loss: {logloss:.4f}')
print(f'Gradient Boosting Accuracy: {accuracy * 100:.4f}%')

Gradient Boosting Log Loss: 0.2599
Gradient Boosting Accuracy: 95.6140%


Let's test it against Scikit's [GradientBoostingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html)

In [28]:
from sklearn.ensemble import GradientBoostingClassifier

boosting_model = GradientBoostingClassifier(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, random_state=1)
boosting_model.fit(X_train, y_train)

ensemble_probs = boosting_model.predict_proba(X_test)[:, 1]  # Probability of class 1
ensemble_preds = boosting_model.predict(X_test)
logloss = log_loss(y_test, ensemble_probs)
accuracy = accuracy_score(y_test, ensemble_preds)
print(f'Gradient Boosting Log Loss: {logloss:.4f}')
print(f'Gradient Boosting Accuracy: {accuracy * 100:.4f}%')

Gradient Boosting Log Loss: 0.1121
Gradient Boosting Accuracy: 95.6140%


# Stacking Methods

Stacking is an ensemble learning technique that combines multiple base models by training a higher-level **meta-model** to aggregate their predictions. Unlike hard or soft voting methods, stacking learns how to best combine base model outputs by leveraging another model that specializes in integrating predictions.

Let $f_1(x), f_2(x), ..., f_m(x)$ represent the $m$ base models. For a given input $x$, the predictions from the base models are:

$$
\mathbf{h}(x) = [f_1(x), f_2(x), ..., f_m(x)]
$$

These predictions form a new feature vector $\mathbf{h}(x)$ for the meta-model $g(\mathbf{h}(x))$. The final stacked prediction is then:

$$
\hat{y} = g(\mathbf{h}(x))
$$


In [29]:
log_reg = LogisticRegression(max_iter=10000, random_state=1)
log_reg.fit(X_train, y_train)
log_reg_preds = log_reg.predict_proba(X_test)[:, 1]

svc_model = SVC(probability=True, random_state=1)
svc_model.fit(X_train, y_train)
svc_preds = svc_model.predict_proba(X_test)[:, 1]

tree_model = DecisionTreeClassifier(random_state=1)
tree_model.fit(X_train, y_train)
tree_preds = tree_model.predict_proba(X_test)[:, 1]

stacked_features = np.column_stack((log_reg_preds, svc_preds, tree_preds))

meta_model = LogisticRegression(max_iter=10000, random_state=1)
meta_model.fit(stacked_features, y_test)
meta_preds = meta_model.predict_proba(stacked_features)[:, 1]
final_preds = meta_model.predict(stacked_features)

logloss = log_loss(y_test, meta_preds)
accuracy = accuracy_score(y_test, final_preds)
print(f'Stacking Log Loss: {logloss:.4f}')
print(f'Stacking Accuracy: {accuracy * 100:.4f}%')

Stacking Log Loss: 0.1162
Stacking Accuracy: 96.4912%


In [30]:
from sklearn.ensemble import StackingClassifier

estimators = [
    ('log_reg', LogisticRegression(max_iter=10000, random_state=1)),
    ('svc', SVC(probability=True, random_state=1)),
    ('tree', DecisionTreeClassifier(random_state=1))
]

stacking_model = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(max_iter=10000, random_state=1))
stacking_model.fit(X_train, y_train)

ensemble_probs = stacking_model.predict_proba(X_test)[:, 1]  # Probability of class 1
ensemble_preds = stacking_model.predict(X_test)
logloss = log_loss(y_test, ensemble_probs)
accuracy = accuracy_score(y_test, ensemble_preds)
print(f'Stacking Log Loss: {logloss:.4f}')
print(f'Stacking Accuracy: {accuracy * 100:.4f}%')

Stacking Log Loss: 0.0985
Stacking Accuracy: 95.6140%


# Conclusion

In this notebook, we have explored several key ensemble learning techniques and their unique strengths:

1. **Bagging:** By training multiple models on different bootstrapped subsets of the data, we learned how to reduce variance and create a robust ensemble that outperforms individual models. The Random Forest is a classic example that demonstrates bagging’s power to handle noise and provide stable predictions.

2. **Boosting:** We saw how boosting sequentially trains weak learners, focusing on correcting the errors of prior models. By optimizing the loss function step-by-step, boosting algorithms—such as AdaBoost and Gradient Boosting—achieve high accuracy and are especially effective at handling complex patterns and imbalanced datasets.

3. **Voting:** Hard and soft voting methods taught us how simple aggregations of multiple models’ predictions can yield improved performance. These approaches illustrate that even without sophisticated training strategies, combining diverse model types can enhance overall accuracy.

4. **Stacking:** Finally, stacking introduced the concept of training a meta-model to learn how best to combine the outputs of multiple base models. By using a higher-level model to integrate predictions, we gain the ability to exploit the complementary strengths of various learning algorithms.

We have seen that ensemble methods are some of the most effective tools in the machine learning toolbox. They consistently deliver top-tier performance on tabular data, often with minimal hyperparameter tuning. By understanding and leveraging these techniques, we can tackle a wide range of predictive tasks with confidence and reliability.
